<div style="text-align: center"><img width=150px src="https://fsharp.org/img/logo/fsharp256.png"></div>

# Introduction to F# #

F# is an open-source and cross-platform language that's supported by the [F# Software Foundation](http://fsharp.org) and a worldwide community of contributors. Microsoft and other companies develop professional tooling for F#.

As a language, F# excels at succinct, correct, and maintainable code, and is scalable from small scripting projects to enterprise-level software. F# is used for [data scripting](https://fsharpforfunandprofit.com/posts/low-risk-ways-to-use-fsharp-at-work-2/), [data science](http://fslab.org), [web programming](http://fsharp.org/guides/web/), and component development. It interoperates with a wide range of software libraries and tools, and you can use all .NET and C# libraries with F#.

F# is well suited for Jupyter notebooks because declarations are ordered in a script-like way. This present notebook, which provides introductory F# example, assumes you are familiar with basic programming structures already. There are also many other [F# learning resources](http://fsharp.org/learn.html), including the[F# Cheatsheet](https://dungpa.github.io/fsharp-cheatsheet/) is a quick guide. 

## F# Basics #

Let's start with some simple arithmetic:

In [ ]:
(12/4 + 5 + 7) * 4 - 18

The following code creates a list of numbers using `[start .. end]`, and assigns to the variable `sampleNumbers` using the `let` statement:

In [ ]:
let sampleNumbers = [ 0 .. 15 ]

sampleNumbers

Next, use `let` to define a function that accepts and returns an integer. When needed, annotate the type of a parameter name using `(argument: type)`. Documentation comments are added using `///`. When calling a function, parenthases are optional:

In [ ]:
/// A function to compute a sample curve
let sampleFunction (x:int) = 
    2*x*x - 5*x + 3

sampleFunction 7

In [ ]:
sampleFunction (7 + 4)

Note that F#, like Python, uses indentation to define blocks. You can find out more about this in the topic [F# syntax: indentation and verbosity](https://fsharpforfunandprofit.com/posts/fsharp-syntax/).

To convert between numeric types `int`, `double`, `int64`, `bigint`, and so on, use conversion functions of the same name:

In [ ]:
sampleFunction (int 3.14 + int 2.79)

Conditionals are expressed using `if...then...else`, and booleans are expressed using `true`, `false`, `&&`, `||` and `not`:

In [ ]:
if 98.0 < 100.0 then 10 else 20

In [ ]:
not false && (true || false)

### Strings, Tuples, Lists and Arrays

String literals use quotes or triple-quotes, and you can concatenate strings with the `+` operator:

In [ ]:
let helloWorld = "Hello" + " " + "world"

helloWorld

In [ ]:
let tripleQuoteString = """A triple-quoted string can contain double quotes " anywhere within it."""

tripleQuoteString

A tuple combines multiple data items into one value. Below is a tuple consisting of an integer, a string, and a double-precision floating point number:

In [ ]:
(1, "fred", 3.1415)

Lists are linear sequences of values of the same type. The code below generates a list containing all the Fridays of the first half of 2017. This example also shows you how to use some of the very extensive .NET libraries, all of which are available to F#. Find out more about the .NET libraries in online resources such as the [.NET Framework API Reference](https://docs.microsoft.com/en-us/dotnet/).

In [ ]:
open System

let fridaysList = 
    [ for month in 1 .. 6 do
        for day in 1 .. DateTime.DaysInMonth(2017, month) do 
          let date = DateTime(2017, month, day)
          if date.DayOfWeek = DayOfWeek.Friday then 
            yield date.ToShortDateString() ]
              
fridaysList              

Arrays are similar to lists but are mutable and are stored as flat data rather than linked lists:

In [ ]:
let lowNumbers = [| 1 .. 200 |]

lowNumbers

Lists, arrays and sequences can be processed using functions. Use the pipeline operator `|>` and a function to process data using `List.map`:

In [ ]:
sampleNumbers 
|> List.map (fun x -> x*x) 

Pipelines can be chained together.  The following pipeline computes the sum of a selection of square numbers:

In [ ]:
let sumOfSelectedSquares = 
    sampleNumbers
    |> List.map (fun x -> x*x) 
    |> List.filter (fun x -> x % 3 = 0)
    |> List.sumBy (fun x -> x * x)
    
sumOfSelectedSquares    

Both lists and arrays can use slicing notation (note that unlike Python, F# slicing notation is inclusive of the upper bound):

In [ ]:
lowNumbers.[0 .. 50]

### Sequences

Sequences are computed on-demand and are specified using the same constructs as lists and arrays.  Here is an infinite sequence which is a random walk. Use `yield!` to return each element of a subsequence:

In [ ]:
let rnd = System.Random()

let rec randomWalk x =
    seq { yield x
          yield! randomWalk (x + rnd.NextDouble() - 0.5) }

let firstValuesOfRandomWalk = 
    randomWalk 5.0 
    |> Seq.truncate 20
    |> Seq.toList
    
firstValuesOfRandomWalk

### Type Definitions

F# is a typed language. The example below defines a record type. Learn more about F# type definitions in online resources such as [F# for Fun and Profit](https://fsharpforfunandprofit.com/posts/overview-of-types-in-fsharp/).

The type definition uses an option value. Option values are any kind of value tagged with either `Some` or `None`. They're used extensively in F# code to represent the cases where many other languages would use null references.

In [ ]:
type ContactCard = 
    { Name     : string
      Phone    : string
      Verified : bool
      ZipCode : string option}            
      
let sampleCard = { Name = "Alf" ; Phone = "(206) 555-0157" ; Verified = false; ZipCode=Some "90210" }

sampleCard

In [ ]:
let showCard (c: ContactCard) = 
    c.Name + " Phone: " + c.Phone + (if not c.Verified then " (unverified)" else "")
    
showCard sampleCard

### UnitsOfMeasure

Code can be annotated with units of measure when using F# arithmetic over numeric types:

In [ ]:
open Microsoft.FSharp.Data.UnitSystems.SI.UnitNames

let raceLength = 1600.0<meter>

[<Measure>]
type mile =
    /// Conversion factor mile to meter: meter is defined in SI.UnitNames
    static member asMeter = 1609.344<meter/mile>

/// Distance expressed using imperial units
let distanceToWalk  = 500.0<mile>          

// Same distanceusing metric system
let distanceToWalkInMeters = distanceToWalk * mile.asMeter   

(raceLength, distanceToWalk, distanceToWalkInMeters)

### Parallel Programming

You can use `Array.Parallel.map` and `Async.Parallel` and related functions to do parallel processing of collections:

In [ ]:
let oneBigArray = [| 0 .. 100000 |]

// Do some CPU intensive computation
let rec computeSomeFunction x = 
    if x <= 2 then 1 
    else computeSomeFunction (x - 1) + computeSomeFunction (x - 2)
       
/// Do a parallel map over a large input array
let computeResults() = oneBigArray |> Array.Parallel.map (fun x -> computeSomeFunction (x % 24))

computeResults()

### Learning More

A range of F# learning topics are available online at [The F# Software Foundation](http://fsharp.org/learn) and also the [Microsoft .NET Docs F# Guide](https://docs.microsoft.com/dotnet/fsharp/).

# Special Features of F# in Azure Notebooks

Azure Notebooks uses [`IfSharp`](http://fsprojects.github.io/IfSharp/) to provide F# Notebooks. This section will take you through some of the nice ways you can combine F# features with the display mechanisms provides by `IfSharp`.

## Referencing NuGet Packages using Paket

There are thousands of well-engineered packages available for use with F#.  Most are provided via [nuget.org](http://nuget.org). You can use the [Paket dependency manager](https://fsprojects.github.io/Paket/) to add references to NuGet packages to your F# Azure Notebook.

In [ ]:
#load "Paket.fsx"

The following code installs the selected packages; it may take a minute to run on first use with a particular kernel.

In [ ]:
Paket.Package
  [ "MathNet.Numerics"
    "MathNet.Numerics.FSharp"
  ]

You can now use these packages from your code using this reference:

In [ ]:
#load "Paket.Generated.Refs.fsx"

For example, you can use the MathNet.Numerics linear algebra libraries:

In [ ]:
open MathNet.Numerics.LinearAlgebra
let m = matrix [[ 1.0; 2.0 ]
                [ 3.0; 4.0 ]]
m.Inverse() 

## Tables

Let's start with using `Util.Table` to display a nicely formatted table:

In [ ]:
type MyType = { FirstName: string; LastName: string }
let records = 
    [|
        { FirstName = "Walter"; LastName = "Harp" }
        { FirstName = "Jeff"; LastName = "Smith" }
        { FirstName = "Ben"; LastName = "Smith" }
        { FirstName = ""; LastName = "Holly" }
    |]

records |> Util.Table

You can also filter the table to display a subset of the data as you display it:

In [ ]:
Util.Table(records, [| "LastName" |])

## Formatting text using LaTeX

The following example composes some LaTeX using `LatexOutput` and the `Util.Math` function provided by `IfSharp`:

In [ ]:
"f(x)" |> Util.Math

Lovely! You've used `Util.Math: string -> LatexOutput` to format LaTeX. The result of your code snippet is implictly passed to the `Display` function and shown in your output. Next we format some operators:

In [ ]:
"\int_0^\infty e^{-x^2} dx \mathrm{\ in\ \LaTeX{}}"
  |> Util.Math

The result of your code snippet is implictly passed to the `Display` function and shown in your output. The `Display` function is provided by `IfSharp`. It takes any object and attempts to display it in your notebook. By default, `Display` already handles many types (including `LatexOutput`), and later we'll see how to extend it to handle any type you need.

You can also call `Display` in your code, in order to display more than one output.


## Charts

We can display charts, using [`XPlot.Plotly`](https://tahahachana.github.io/XPlot/plotly.html):

In [ ]:
#load "XPlot.Plotly.Paket.fsx"
#load "XPlot.Plotly.fsx"
open XPlot.Plotly

For example:

In [ ]:
Heatmap(z = [[1; 20; 30]; [20; 1; 60]; [30; 60; 1]])
    |> Chart.Plot
    |> Chart.WithLayout (Layout(title = "Simple Heatmap"))

In [ ]:
["giraffes", 20; "orangutans", 14; "monkeys", 23]
    |> Chart.Bar
    |> Chart.WithLayout (Layout(title = "Basic Bar Chart"))
    |> Chart.WithHeight 300
    |> Chart.WithWidth 400

Note that we had to `#load` two helper scripts in order to load the assemblies we need and to enable `Display` to show our charts. The first downloads and installs the required Paket packages, and the second sets up `Display` support.

### Other charting libraries

You can use any charting library on [nuget.org](http://nuget.org) in a notebook. For example, the code below uses [Angara.Chart](http://predictionmachines.github.io/Angara.Chart) to render charts:

In [ ]:
#load "Angara.Charting.Paket.fsx"
#load "Angara.Charting.fsx"

In [ ]:
let x = [| for i in 0..99 -> float(i) / 10.0 |]
let y = x |> Array.map (fun x -> sin x)
let z = x |> Array.map (fun x -> cos x)

[ Angara.Charting.Plot.line(x, y); Angara.Charting.Plot.markers(x, z) ] |> Angara.Charting.Chart.ofList

## Computed inline HTML 

You can also format HTML and display it by producing values of a special type ``HtmlOutput`` (a record type with one field ``Html``):

In [ ]:
let html x = { Html = x }
 
html """<div style="background-color: #007fff; font-weight: bold;">Would you like some toast?</div>"""

// Exercise - try video content:
//  <video width="480" controls poster="https://archive.org/download/WebmVp8Vorbis/webmvp8.gif" >
//    <source src="https://archive.org/download/WebmVp8Vorbis/webmvp8.webm" type="video/webm">
//    <source src="https://archive.org/download/WebmVp8Vorbis/webmvp8_512kb.mp4" type="video/mp4">
//    <source src="https://archive.org/download/WebmVp8Vorbis/webmvp8.ogv" type="video/ogg">
//    Your browser doesn't support HTML5 video tag.
//  </video>"""

This type renders its content directly as HTML.

Certain sites of content are whitelisted for use from Azure Notebooks.  At the time of writing these include `raw.githubusercontent.com`, `github.com`, `onedrive.live.com` and a range of Azure services. You can include content from these sites directly through `Util.Url`:

In [ ]:
"https://github.com/fsharp/fsfoundation/raw/gh-pages/img/logo/fsharp256.png"
  |> Util.Url

> **Note**: Evaluation uses a REPL
>
> Notice that we used a value from an earlier snippet in a later one. Your entire notebook is a single program running in an `fsi` REPL, so you accumulate data and functions as you go. The caveat is that code cells aren't reevaluated unless you specifically run them. If you go back and edit an earlier snippet, then, you can reference objects that are defined in later code cells! If you reevaluate your entire notebook, this causes errors.
>
> Just remember that your notebook is a single REPL instance, and you won't be surprised by this behavior.

## Custom `Display` Printers

The `Display` function can show many things in your F# Azure Notebook already, but you can extend it to display any type in any way that suits you. To do this, use `IfSharp.Kernel.App.AddDisplayPrinter` to add a function that accepts your type and returns a record that includes a `ContentType: string` (which is interpreted as a MIME Content-Type) and a `Data: string`, which is the content that will be interpreted by your browser as the specified Content-Type.

Often this will be "text/html" and some HTML. Here's an example:

In [ ]:
open IfSharp.Kernel.App

type Person = 
    { Name: string }

AddDisplayPrinter (fun (person: Person) -> { ContentType = "text/html"; Data = "<b><i>Ahoy " + person.Name + "!</i></b>" })

After this the Person type will display in the requiested way:

In [ ]:
let samplePerson = { Name = "Walter" }

samplePerson

# Using JavaScript to display dynamic output

You can also use existing browser display technologies directly in your Azure Notebook. Start by loading D3:

In [ ]:
@"<script src=""https://d3js.org/d3.v3.min.js""></script>" |> Util.Html |> Display

Now define a few F# types and functions that will let you write D3 output as structured F# data. These types are pretty simple, and don't cover everything D3 can do, but it shows how F# can interact with JavaScript directly in your notebook.

In [ ]:
type D3Op = {
    Op: string
    Style: List<string * string>
    Attr: List<string * int>
    On: List<string * List<D3Op>>
}

type D3 = List<D3Op>

let d3op = {Op = ""; Style = []; Attr = []; On = []}

let mapConcat s f xs = 
    xs |> List.map f |> String.concat s

let rec D3OpPrinter (op: D3Op) =
    sprintf 
        "%s%s%s%s"
        (if op.Op.Length > 0 then
            sprintf "\t.append(\"%s\")\n" op.Op
        else
            "")
        (op.Style |> mapConcat "\t" (fun (k, v) -> sprintf ".style(\"%s\", \"%s\")\n" k v ) )
        (op.Attr |> mapConcat "\t" (fun (k, v) -> sprintf ".attr(\"%s\", %d)\n" k v))
        (op.On
            |> mapConcat "\t" (fun (k, v) ->
                sprintf 
                    ".on(\"%s\",\n\tfunction(){\n\td3.select(this)\n%s\t})\n"
                    k
                    (v |> mapConcat "" D3OpPrinter)
                ))

let D3Printer (d3: D3) =
    sprintf 
        """
<div id="viz"></div>
<script type="text/javascript">

d3.select("#viz")
%s
"""
        (d3 |> mapConcat "" D3OpPrinter)

App.AddDisplayPrinter (fun (d3: D3) ->
  { ContentType = "text/html"
    Data = D3Printer d3 }
  )

Now use D3 to render a circle that changes color when you put your mouse over it, by writing structured data:

In [ ]:
[
    {d3op with 
        Op = "svg"
        Attr = [("width", 100); ("height", 100)] }
    {d3op with
        Op = "circle"
        Style = [("stroke", "grey"); ("fill", "white")]
        Attr = [("r", 40); ("cx", 50); ("cy", 50)]
        On = 
          [ ("mouseover", [{d3op with Style = [("fill", "blue")]}])
            ("mouseout", [{d3op with Style = [("fill", "white")]}]) ] }
]

## Additional resources

The [F# Language Reference](https://docs.microsoft.com/dotnet/articles/fsharp/) is a reference for the F# language, and the [F# Guide](https://docs.microsoft.com/dotnet/articles/fsharp/) covers general topics.

[F# on Azure](https://docs.microsoft.com/dotnet/articles/fsharp/using-fsharp-on-azure/) is a technical guide for using F# in conjunction with a range of Azure services.